### Code used to clean and reformat weather data from
- MAC Season 4
- MAC Season 6
- KSU (Ashland Bottoms)
- Clemson (Pee Dee Research and Education Center, Florence)

### Weather parameters found in all raw data
- Date 
- Day of year
- Temperature minimum
- Temperature maximum
- Temperature mean
- Accumulated growing degree days (gdd)
- Relative humidity minimum
- Relative humidity maximum
- Relative humidity mean
- Vapor pressure deficit
- Precipitation
- Cumulative precipitation
- First water deficit treatment (for MAC season 4)
- Second water deficit treatment (for MAC season 4)

### Season dates
- MAC Season 4
    - Planting: 2017-04-20, Day 110
    - Last Day of Harvest: 2017-09-16, Day 259 
- MAC Season 6
    - Planting: 2018-04-25, Day 115
    - Harvest: 2018-08-01, Day 213
- KSU
    - Planting: 2016-06-17, Day 169
    - Harvest: 2016-10-21, Day 295
- Clemson
    - Planting: 2014-05-06, Day 126
    - Latest date in Clemson trait data: 2014-10-15, Day 288

#### See README for information on units, accessing raw data, and downloading processed data

In [ ]:
import datetime
import numpy as np
import pandas as pd

##### Functions

In [ ]:
def calculate_vpd(temp_avg, rh_avg):
    es = (6.11 * np.exp((2500000/461) * (1/273 - 1/(273 + temp_avg))))
    vpd = (((100 - rh_avg)/1000) * es)
    return vpd

In [ ]:
def save_to_csv(list_of_dfs, list_of_output_filenames):
    for i,j in zip(list_of_dfs, list_of_output_filenames):
        i.to_csv(j, index=False)

##### URLs for downloading raw weather data

In [ ]:
s4_url = 'https://de.cyverse.org/dl/d/7D6C8FD6-EF77-437C-89E6-412EA8C3EEC6/mac_weather_station_raw_daily_2017.csv'
s6_url = 'https://de.cyverse.org/dl/d/233C21D5-1306-4028-9CF9-FF4AF0EAC405/mac_weather_station_raw_daily_2018.csv'
ksu_hourly_url = 'https://de.cyverse.org/dl/d/D80C07D7-5F68-4C86-B15A-9BAAF472D3A4/ksu_hourly_weather.csv'
ksu_daily_url = 'https://de.cyverse.org/dl/d/64805E3B-0460-4AA1-8D8A-2D7246E05B35/ashland_bottoms_daily_weather_2016.csv' 
clemson_temps_url = 'https://de.cyverse.org/dl/d/19836AB5-9223-4CBA-B56E-46272CACF5A3/clemson_temps_daily.csv'
clemson_rh_url = 'https://de.cyverse.org/dl/d/353F5386-4D88-45A9-A4F6-6FC66C64C7E8/clemson_rh_daily.csv'

### A. MAC Season 4

In [ ]:
s4_0 = pd.read_csv(s4_url)
print(s4_0.shape)
s4_0.head(3)

##### Slice for season dates only and add date column

In [ ]:
s4_1 = s4_0.loc[(s4_0.day_of_year >= 110) & (s4_0.day_of_year <= 259)]
season_4_date_range = pd.date_range(start='2017-04-20', end='2017-09-16')

In [ ]:
s4_2 = s4_1.copy()
s4_2['date'] = season_4_date_range
# s4_2.tail(3)

##### Add growing degree days

In [ ]:
s4_3 = s4_2.copy()
s4_3['daily_gdd'] = (((s4_3['air_temp_max'] + s4_3['air_temp_min'])) / 2) - 10

In [ ]:
s4_4 = s4_3.copy()
s4_4['gdd'] = np.rint(np.cumsum(s4_4['daily_gdd']))

In [ ]:
s4_5 = s4_4.drop(labels='daily_gdd', axis=1)
# print(s4_5.shape)
# s4_5.head()

In [ ]:
s4_6 = s4_5.copy()
s4_6['cum_precip'] = np.cumsum(s4_6.precip_total)

##### Add columns for water-deficit stress treatment 
- First water-deficit stress treatment: 2017-08-01 through 2017-08-14
- Second water-deficit stress treatment: 2017-08-15 through 2017-08-30

In [ ]:
first_treatment_dates = pd.date_range(start='2017-08-01', end='2017-08-14')
second_treatment_dates = pd.date_range(start='2017-08-15', end='2017-08-30')
season_dates = s4_6.date.values

In [ ]:
first_treatment_col = []

for d in season_dates:
    if d in first_treatment_dates:
        first_treatment_col.append(True)    
    else: 
        first_treatment_col.append(False)

In [ ]:
second_treatment_col = []

for d in season_dates:
    if d in second_treatment_dates:
        second_treatment_col.append(True)
    else:
        second_treatment_col.append(False)

In [ ]:
s4_7 = s4_6.copy()

s4_7['first_water_deficit_treatment'] = first_treatment_col
s4_7['second_water_deficit_treatment'] = second_treatment_col

##### Drop columns

In [ ]:
# s4_7.columns

In [ ]:
s4_cols_to_keep = ['day_of_year', 'air_temp_max', 'air_temp_min', 'air_temp_mean', 'rh_max', 'rh_min', 'rh_mean', 'vpd_mean', 
                'precip_total', 'date', 'gdd', 'cum_precip', 'first_water_deficit_treatment', 
                'second_water_deficit_treatment']

s4_8 = pd.DataFrame(data=s4_7, columns=s4_cols_to_keep)
# print(s4_8.shape)
# s4_8.head()

##### Rename some columns

In [ ]:
s4_new_col_names = ['day_of_year', 'temp_max', 'temp_min', 'temp_mean', 'rh_max', 'rh_min', 'rh_mean', 'vpd_mean',
                   'precip', 'date', 'gdd', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

s4_9 = s4_8.copy()
s4_9.columns = s4_new_col_names
# print(s4_9.shape)
# s4_9.tail()

##### Reorder columns

In [ ]:
s4_new_col_order = ['date', 'day_of_year', 'temp_min', 'temp_max', 'temp_mean', 'gdd', 'rh_min', 'rh_max', 'rh_mean', 
                    'vpd_mean', 'precip', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

In [ ]:
s4_10 = s4_9[s4_new_col_order]
# print(s4_10.shape)
# s4_10.tail(3)

##### Round numeric columns to two decimal points, except for `gdd` which was already rounded to the nearest integer

In [ ]:
s4_cols_to_round = ['temp_min', 'temp_max', 'temp_mean', 'rh_min', 'rh_max', 'rh_mean', 'vpd_mean', 'precip',
                    'precip_cumulative']

s4_11 = s4_10.copy()
s4_11[s4_cols_to_round] = s4_10[s4_cols_to_round].round(2)
print(s4_11.shape)
s4_11.head(3)

### B. MAC Season 6

In [ ]:
s6_0 = pd.read_csv(s6_url)
print(s6_0.shape)
s6_0.head(3)

##### Slice for season dates only and add date column

In [ ]:
s6_1 = s6_0.loc[(s6_0.day_of_year >= 115) & (s6_0.day_of_year <= 213)]
season_6_date_range = pd.date_range(start='2018-04-25', end='2018-08-01')

In [ ]:
s6_2 = s6_1.copy()
s6_2['date'] = season_6_date_range
# s6_2.tail(3)

##### Add growing degree days

In [ ]:
s6_3 = s6_2.copy()
s6_3['daily_gdd'] = (((s6_3['air_temp_max'] + s6_3['air_temp_min'])) / 2) - 10

In [ ]:
s6_4 = s6_3.copy()
s6_4['gdd'] = np.rint(np.cumsum(s6_4['daily_gdd']))

In [ ]:
s6_5 = s6_4.drop(labels='daily_gdd', axis=1)
# print(s6_5.shape)
# s6_5.head()

In [ ]:
s6_6 = s6_5.copy()
s6_6['cum_precip'] = np.cumsum(s6_6.precip_total)

##### Add first and second water deficit treatment columns
- All values will be `False`

In [ ]:
s6_7 = s6_6.copy()
s6_7['first_water_deficit_treatment'] = False
s6_7['second_water_deficit_treatment'] = False
# s6_7.head()

##### Drop some columns

In [ ]:
s6_cols_to_keep = ['day_of_year', 'air_temp_max', 'air_temp_min', 'air_temp_mean', 'rh_max', 'rh_min', 'rh_mean', 'vpd_mean', 
                   'precip_total', 'date', 'gdd', 'cum_precip', 'first_water_deficit_treatment', 
                   'second_water_deficit_treatment']
s6_8 = pd.DataFrame(data=s6_7, columns=s6_cols_to_keep)
# print(s6_8.shape)
# s6_8.head()

##### Rename columns

In [ ]:
s6_new_col_names = ['day_of_year', 'temp_max', 'temp_min', 'temp_mean', 'rh_max', 'rh_min', 'rh_mean', 'vpd_mean',
                   'precip', 'date', 'gdd', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

s6_9 = s6_8.copy()
s6_9.columns = s6_new_col_names
# print(s6_9.shape)
# s6_9.tail()

##### Reorder columns

In [ ]:
s6_new_col_order = ['date', 'day_of_year', 'temp_min', 'temp_max', 'temp_mean', 'gdd', 'rh_min', 'rh_max', 'rh_mean', 
                    'vpd_mean', 'precip', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

In [ ]:
s6_10 = s6_9[s6_new_col_order]
# print(s6_10.shape)
# s6_10.tail(3)

##### Round numeric columns to two decimal points, except for `gdd` which was already rounded to the nearest integer

In [ ]:
s6_cols_to_round = ['temp_min', 'temp_max', 'temp_mean', 'rh_min', 'rh_max', 'rh_mean', 'vpd_mean', 'precip',
                    'precip_cumulative']

s6_11 = s6_10.copy()
s6_11[s6_cols_to_round] = s6_10[s6_cols_to_round].round(2)
print(s6_11.shape)
s6_11.head(3)

### C. KSU Hourly
- to calculate daily vapor pressure deficit mean values

In [ ]:
kh_0 = pd.read_csv(ksu_hourly_url)
print(kh_0.shape)
kh_0.head(3)

In [ ]:
kh_1 = kh_0.iloc[2:]
# kh_1.head(3)

In [ ]:
kh_2 = kh_1.copy()
kh_2['AirTemperature'] = pd.to_numeric(kh_2['AirTemperature'], errors='coerce')

In [ ]:
kh_3 = kh_2.copy()
kh_3['RelativeHumidity'] = pd.to_numeric(kh_3['RelativeHumidity'], errors='coerce')
# print(kh_3.shape)
# print(kh_3.dtypes)
# kh_3.head()

In [ ]:
kh_4 = kh_3.copy()
kh_4['vpd_mean'] = calculate_vpd(kh_4['AirTemperature'], kh_4['RelativeHumidity'])
# print(kh_4.shape)
# kh_4.head()

In [ ]:
# kh_4.isnull().sum()

In [ ]:
kh_5 = kh_4.dropna(how='any', axis=0)
# print(kh_5.shape)

In [ ]:
just_dates = []

for timestamp in kh_5['Timestamp'].values:
    date = timestamp[:10]
    just_dates.append(date)

In [ ]:
kh_6 = kh_5.copy()
kh_6['date'] = just_dates

# kh_6.tail()

In [ ]:
kh_7 = kh_6.groupby(['date'], as_index=False)['vpd_mean'].mean()
# print(kh_7.shape)
# kh_7.head(3)

##### Add minimum and maximum relative humidity

In [ ]:
kh_8 = kh_6.groupby(['date']).agg(rh_min=('RelativeHumidity', 'min'), rh_max=('RelativeHumidity', 'max')).reset_index()
# print(kh_8.shape)
# kh_8.head(3)

In [ ]:
kh_9 = kh_7.merge(kh_8, how='left', left_on='date', right_on='date')
print(kh_9.shape)
kh_9.tail(3)

### D. Kansas Daily

In [ ]:
ksu_0 = pd.read_csv(ksu_daily_url)
print(ksu_0.shape)
ksu_0.head(3)

In [ ]:
ksu_1 = ksu_0.iloc[2:]

In [ ]:
ksu_2 = ksu_1.merge(kh_9, how='left', left_on='Timestamp', right_on='date')
# print(ksu_2.shape)
# ksu_2.head()

##### Convert temperature, relative humidity, and precipitation values to numeric

In [ ]:
# ksu_2.dtypes

In [ ]:
ksu_numeric_cols = ['AirTemperature', 'AirTemperature.1', 'RelativeHumidity', 'Precipitation']
ksu_3 = ksu_2.copy()
ksu_3[ksu_numeric_cols] = ksu_3[ksu_numeric_cols].apply(pd.to_numeric, errors='coerce')
# print(ksu_3.shape)
# print(ksu_3.dtypes)
# ksu_3.tail()

##### Add growing degree days

In [ ]:
ksu_4 = ksu_3.copy()
ksu_4['daily_gdd'] = (((ksu_4['AirTemperature'] + ksu_4['AirTemperature.1'])) / 2) - 10

In [ ]:
ksu_5 = ksu_4.copy()
ksu_5['gdd'] = np.rint(np.cumsum(ksu_5['daily_gdd']))

In [ ]:
ksu_6 = ksu_5.drop(labels='daily_gdd', axis=1)
# print(ksu_6.shape)
# ksu_6.head()

##### Add cumulative precipitation

In [ ]:
ksu_7 = ksu_6.copy()
ksu_7['precip_cumulative'] = np.cumsum(ksu_7['Precipitation'])

##### Add first and second water deficit treatment columns
- All values will be `False`

In [ ]:
ksu_8 = ksu_7.copy()
ksu_8['first_water_deficit_treatment'] = False
ksu_8['second_water_deficit_treatment'] = False
# ksu_8.head()

##### Drop some columns

In [ ]:
ksu_cols_to_keep = ['AirTemperature', 'AirTemperature.1', 'RelativeHumidity', 'Precipitation', 'date', 'vpd_mean', 
                    'rh_min', 'rh_max', 'gdd', 'precip_cumulative', 'first_water_deficit_treatment', 
                    'second_water_deficit_treatment']
ksu_9 = pd.DataFrame(data=ksu_8, columns=ksu_cols_to_keep)
# print(ksu_9.shape)
# ksu_9.head()

##### Add `day_of_year` and `temp_mean` columns

In [ ]:
days_of_year = [i for i in range(169, 296)]

In [ ]:
ksu_10 = ksu_9.copy()
ksu_10['day_of_year'] = days_of_year

In [ ]:
ksu_11 = ksu_10.copy()
ksu_11['temp_mean'] = ksu_11[['AirTemperature', 'AirTemperature.1']].mean(axis=1)
# print(ksu_11.shape)
# ksu_11.head()

##### Rename columns

In [ ]:
ksu_new_col_names = ['temp_max', 'temp_min', 'rh_mean', 'precip', 'date', 'vpd_mean', 'rh_min', 'rh_max', 'gdd',
                    'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment',
                    'day_of_year', 'temp_mean']
ksu_12 = ksu_11.copy()
ksu_12.columns = ksu_new_col_names
# print(ksu_12.shape)
# ksu_12.tail()

##### Reorder columns

In [ ]:
ksu_new_col_order = ['date', 'day_of_year', 'temp_min', 'temp_max', 'temp_mean', 'gdd', 'rh_min', 'rh_max', 'rh_mean', 
                    'vpd_mean', 'precip', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

In [ ]:
ksu_13 = ksu_12[ksu_new_col_order]

##### Round numeric columns to two decimal points, except for `gdd` which was already rounded to the nearest integer

In [ ]:
ksu_cols_to_round = ['temp_min', 'temp_max', 'temp_mean', 'rh_min', 'rh_max', 'rh_mean', 'vpd_mean', 'precip',
                    'precip_cumulative']

ksu_14 = ksu_13.copy()
ksu_14[ksu_cols_to_round] = ksu_14[ksu_cols_to_round].round(2)
print(ksu_14.shape)
ksu_14.head(3)

### E. Clemson Weather

In [ ]:
clemson_0 = pd.read_csv(clemson_temps_url)
print(clemson_0.shape)
clemson_0.head(3)

In [ ]:
clemson_1 = pd.read_csv(clemson_rh_url)
print(clemson_1.shape)
clemson_1.head(3)

##### Add dates in string format to `clemson_0` dataframe

In [ ]:
dates_2014 = clemson_1['DateTime'].values
clemson_2 = clemson_0.copy()
clemson_2['date'] = dates_2014
# print(clemson_2.shape)
# clemson_2.head(3)

##### Merge two dataframes

In [ ]:
clemson_3 = clemson_2.merge(clemson_1, how='left', left_on='date', right_on='DateTime')
print(clemson_3.shape)
clemson_3.tail(3)

##### Slice for season dates

In [ ]:
clemson_4 = clemson_3.loc[(clemson_3['yday'] >= 126) & (clemson_3['yday'] <= 288)]
print(clemson_4.shape)

##### Add growing degree days

In [ ]:
clemson_5 = clemson_4.copy()
clemson_5['daily_gdd'] = (((clemson_5['tmax (deg c)'] + clemson_5['tmin (deg c)'])) / 2) - 10

In [ ]:
clemson_6 = clemson_5.copy()
clemson_6['gdd'] = np.rint(np.cumsum(clemson_6['daily_gdd']))

In [ ]:
clemson_7 = clemson_6.drop(labels='daily_gdd', axis=1)
print(clemson_7.shape)
# clemson_7.head()

##### Add cumulative precipitation

In [ ]:
clemson_8 = clemson_7.copy()
clemson_8['precip_cumulative'] = np.cumsum(clemson_8['prcp (mm/day)'])

##### Add first and second water deficit treatment columns
- All values will be `False`

In [ ]:
clemson_9 = clemson_8.copy()
clemson_9['first_water_deficit_treatment'] = False
clemson_9['second_water_deficit_treatment'] = False
# clemson_9.head()

##### Drop some columns

In [ ]:
clemson_cols_to_keep = ['yday', 'prcp (mm/day)', 'tmax (deg c)', 'tmin (deg c)', 'date', '(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31', 
                        '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31', 'gdd', 
                        'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']
clemson_10 = pd.DataFrame(data=clemson_9, columns=clemson_cols_to_keep)
# print(clemson_10.shape)
# clemson_10.head()

##### Change column names

In [ ]:
clemson_11 = clemson_10.rename({'yday': 'day_of_year', 'prcp (mm/day)': 'precip', 'tmax (deg c)': 'temp_max', 
                                'tmin (deg c)': 'temp_min', '(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'rh_min', 
                                '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'rh_max',}, axis=1)
# clemson_11.tail(3)

##### Columns to calculate and add
- `temp_mean`
- `rh_mean`
- `vpd_mean`

In [ ]:
clemson_12 = clemson_11.copy()
clemson_12['temp_mean'] = clemson_12[['temp_max', 'temp_min']].mean(axis=1)
# print(clemson_12.shape)
# clemson_12.head()

In [ ]:
clemson_13 = clemson_12.copy()
clemson_13['rh_mean'] = clemson_13[['rh_max', 'rh_min']].mean(axis=1)
# print(clemson_13.shape)
# clemson_13.tail(3)

In [ ]:
clemson_14 = clemson_13.copy()
clemson_14['vpd_mean'] = calculate_vpd(clemson_14['temp_mean'], clemson_14['rh_mean'])

##### Reorder columns

In [ ]:
clemson_new_col_order = ['date', 'day_of_year', 'temp_min', 'temp_max', 'temp_mean', 'gdd', 'rh_min', 'rh_max', 'rh_mean', 
                         'vpd_mean', 'precip', 'precip_cumulative', 'first_water_deficit_treatment', 'second_water_deficit_treatment']

In [ ]:
clemson_15 = clemson_14[clemson_new_col_order]

##### Round numeric columns to two decimal points, except for `gdd` which was already rounded to the nearest integer

In [ ]:
clemson_cols_to_round = ['temp_min', 'temp_max', 'temp_mean', 'rh_min', 'rh_max', 'rh_mean', 'vpd_mean', 'precip', 
                         'precip_cumulative']

clemson_16 = clemson_15.copy()
clemson_16[clemson_cols_to_round] = clemson_16[clemson_cols_to_round].round(2)
print(clemson_16.shape)
clemson_16.head(3)

##### Save weather data to .csv format

In [ ]:
list_of_dfs = [s4_11, s6_11, ksu_14, clemson_16]
list_of_output_filenames = ['data/weather/mac_season_4_weather.csv', 'data/weather/mac_season_6_weather.csv',
                           'data/weather/ksu_weather.csv', 'data/weather/clemson_weather.csv']

save_to_csv(list_of_dfs, list_of_output_filenames)